In [ ]:
import warnings
warnings.simplefilter('ignore', UserWarning)

from market_timing import MarketTiming
from qdb import QdbApi
q = QdbApi()

## 指标：北向资金净买入

$$
X_t:= NorthCap(t_i)
$$

使用过去50个交易日作为滑动窗口进行Z-score标准化，得到最终输入信号$fcst$
$$
fcst = \{fcst_t,\ t\in T\}:= \{f(X_t),\ t\in T\}
= \left\{ \frac{X_t - \bar{x}_t}{\sigma_{t}}, t\in T \right\}
$$
$\bar{x}_t$和$\sigma_{t}$为过去50个交易日$[t-49, t]$的均值和标准差

In [ ]:
mt_model = MarketTiming(history_start_date='2015-07-01')

north_cap_sgn = q.get_macro_data(macro_ids=["north_cap"]).set_index("data_date")["value_"]
mt_model.signal_input(north_cap_sgn)

## 内积评估
定义
$$ret(t+h) = \left.\left (\frac{Close(t+h)}{Open(t+1)} - 1\right )\right/ h $$
表示未来$h$天股指的平均日涨跌幅和
$$ret(t+h_1, t+h_2) = \left.\left(\frac{Close(t+h_2)}{Open(t+h_1)} - 1\right)\right/ (h_2-h_1+1)$$
表示从未来$h_1$天到未来$h_2$天股指的涨跌幅。再使用不同的时间期限分别计算内积
$$
fcst^T \cdot [ ret_{1}, ret_{5}, ret_{10}, ret_{21}, ret_{63} ]
$$
和
$$
fcst^T \cdot [ ret_{1}, ret_{2, 5}, ret_{6, 10}, ret_{11, 21}, ret_{22, 63} ]
$$
生成累积的内积序列，分别作图比较

In [ ]:
mt_model.dot_product_analysis()

## 与主要因子相关性

In [ ]:
mt_model.corr_analysis()

## 回测


- 做多：$t$日触发$fcst_{t} > 1$时，对$t+1$日的股指进行做多操作（$pos_{t+1} = 1$），$t+2$日恢复平仓（$pos_{t+2} = 0$）
- 做空：$t$日触发$fcst_{t} < 1$时，对从$t+22$日到$t+63$日的股指开始进行做空操作（$pos = -1$），$t+64$日恢复平仓
- 做空操作期间，如果触发做多信号，则立即转为做多

股指隔夜涨跌幅
$$overnight\_ret_t = \frac{Open_t}{Close_{t-1}} - 1$$
股指白天涨跌幅
$$daytime\_ret_t = \frac{Close_t}{Open_{t}} - 1$$
因此$t$日的策略收益$stgy\_ret_t$由隔夜收益和白天收益两部分组成
$$stgy\_ret_t = (1 + pos_{t-1} \cdot overnight\_ret_t) (1 + pos_{t} \cdot daytime\_ret_t) - 1$$

## 业绩评估

In [ ]:
mt_model.backtest(
    long_threshold=1, long_start=1, long_period=1,
    short_threshold=-1, short_start=11, short_period=10,
    mode='history'
)

In [ ]:
mt_model.backtest(
    long_threshold=1, long_start=1, long_period=1,
    short_threshold=-1, short_start=11, short_period=10,
    mode='test'
)

In [ ]:
mt_model.backtest(
    long_threshold=1, long_start=1, long_period=1,
    short_threshold=-1, short_start=11, short_period=10,
    mode='all'
)

In [ ]:
盈亏比
盈亏分布曲线
收益率周月度效益分布（+节假日前后）
换手率
主要因子相关性（动量因子-过去20）
指数 线性回归 alpha beta


两套策略 
 - 每天调仓
- 